In [91]:
import time
import numpy as np

def extract_data(halo, rscale=1.25, radius='rvir'):
    xc_tmp0 = halo['x']
    yc_tmp0 = halo['y']
    zc_tmp0 = halo['z']
       
    #rr_tmp0 = min([halo[radius] * rscale, 0.0002]) 
    rr_tmp0 = max([halo[radius] * rscale, 0.0001])
    # arbitrary! < 20kpc/h
    
    # When merger occurs, larger radius is likely to include 
    # companion galaxy resulting center to be in the middle of nowhere.
    # If you want a larger galaxy, # increase rgal_tmp instead. 
    #        
    # xx is easier to search for than x.

    if star_all is not None:
        ind_s = np.where((star_all['x'] - xc_tmp0)**2 + (star_all['y'] - yc_tmp0)**2 
                        + (star_all['z'] - zc_tmp0)**2 < rr_tmp0**2)[0]
    if dm_all is not None:
        ind_d = np.where((dm_all['x'] - xc_tmp0)**2 + (dm_all['y'] - yc_tmp0)**2 
                        + (dm_all['z'] - zc_tmp0)**2 < rr_tmp0**2)[0]
    if cell_all is not None:
        ind_c = np.where((cell_all['x'] - xc_tmp0)**2 + (cell_all['y'] - yc_tmp0)**2 
                        + (cell_all['z'] - zc_tmp0)**2 < rr_tmp0**2)[0]
    else:
        return star_all[ind_s], dm_all[ind_d], None
        
    return star_all[ind_s], dm_all[ind_d], cell_all[ind_c]



def mk_gal(halodata, info, i, idx,
           save=False, rscale=1.5, verbose=False, galaxy_plot_dir='./',
           rscale_lambda=3.0, npix_lambda=50, npix=400, galaxy_plot=False,
           method_com=2, mstar_min=5e9):
    
    from galaxy.galaxy2 import Galaxy

    rscale_extract = 1.1
    star, dm, cell = extract_data(h.data[i], rscale=rscale_extract, radius='rvir')
    if mstar_min > 0.:
        if sum(star['m']) * info.msun < mstar_min:
            print("(1)Not enough stars: {:.2f} Msun".format(sum(star['m']) * info.msun))
            print("Aborting... \n")
            print(" Not a good galaxy")
            out_q.put(gal_out)
            return
               
    #Create galaxy ----------------------------------------------
    gal = Galaxy(halodata, radius_method='eff', info=info)
    #print(i, time.time() - t, "seconds ---2")
    good_gal = gal.mk_gal_from_gal(star, dm, cell,
                        mstar_min=mstar_min,
               rscale=min([rscale,rscale_extract]), verbose=verbose, method_com=method_com)

    
    gal.cal_lambda_r(npix=npix_lambda, method=1, rscale=rscale_lambda) # calculate within 1.0 * reff    

    gal.plot_gal(fn_save = galaxy_plot_dir + str(nout).zfill(3) \
                         + "_" + str(idx).zfill(7) + "_"  \
                         + str(gal.id) + ".png", ioff=True)
                             
    return gal
   

In [92]:
def photometry(img, nbin, npix_img, l_img):
    """
    What does it do? 
    
    """
    arr=np.recarray((nbin,), dtype=[("eps", np.float),
                 ("mjr", np.float), ("pa", np.float),
                 ("xpos", np.float), ("ypos", np.float)])

    for i, frac in enumerate(np.linspace(0.005, 0.15, num=nbin, endpoint=True)):
        f = mge.find_galaxy.find_galaxy(data, quiet=True, plot=True, mask_shade=False, fraction=frac)
        arr.mjr[i] = f.majoraxis
        arr.eps[i] = f.eps
        arr.pa[i] = f.theta
        arr.xpos[i] = f.xpeak
        arr.ypos[i] = f.ypeak

    # convert units
    arr.mjr = arr.mjr * 3.5 / npix_img * l_img # in kpc.
    arr.xpos = (arr.xpos / npix_img - 0.5) * l_img # in kpc.
    arr.ypos = (arr.ypos / npix_img - 0.5) * l_img # in kpc.
        
    return arr

def sigmap_vmap(self, npix, ind_ok=None):
    if ind_ok is None:
        rr = self.reff * self.rscale_lambda

        ind_ok = np.where((abs(self.star["x"]) <= rr) &
                          (abs(self.star["y"]) <= rr) &
                          (abs(self.star["z"]) <= rr) )[0]
        
        print(("\n" "Calculating Lambda_r for {} particles "
           "inside {:.3f}kpc, or {}Reff".format(len(ind_ok), rr, rscale)))

    x = self.star['x'][ind_ok]
    y = self.star['y'][ind_ok]
    m = self.star['m'][ind_ok]
    vz = self.star['vz'][ind_ok]

    # NGP charge assignment
    nx = npix
    ny = npix

    # distance in pixel unit.
    dist2d=np.zeros((nx, ny), dtype=float)
    for i in range(nx):
        for j in range(ny):
            dist2d[i][j]= np.sqrt((0.5 + i - nx/2)**2 + (0.5 + j - ny/2)**2)

    dist1d = dist2d.ravel()

    # fix center explicitely.
    # 0.5 * (min + max) != center
    xx = (x + rr) / rr * 0.5 * nx
    yy = (y + rr) / rr * 0.5 * ny

    ngx = np.clip(np.fix(xx), 0, nx-1)
    ngy = np.clip(np.fix(yy), 0, ny-1)

    indices = ngx + ngy * nx

    mmap = np.zeros(nx * ny, dtype=float)
    for i, ind in enumerate(indices):
        mmap[ind] += m[i]

    dx = (2 * rr) / npix2
    mmap = mmap / (dx*dx)
    self.mmap = mmap.reshape(nx, ny)

    vmap = np.zeros(nx * ny, dtype=float)
    # mass-weighted sigma         
    sigmap=np.zeros(nx * ny, dtype=float)
    for i in range(nx * ny):
        ind = np.where(indices == i)[0]
        if len(ind) > 0:
            sigmap[i] = self.weighted_std(vz[ind], m[ind])
            vmap[i] = np.average(vz[ind], weights=m[ind])
        else:
            sigmap[i] = 0
            vmap[i] = 0

    return sigmap.reshape(nx, ny), vmap.reshape(nx, ny)

In [94]:
#%%

"""
The processing pool needs to be instantiated in the main 
thread of execution. 
"""
import load
import matplotlib.pyplot as plt
from tree import tmtree, treemodule
import tree.ctutils as ctu
import utils.sampling as smp
import tree.halomodule as hmo 
import os
import pickle
hydro = True
is_gal = True

wdir = '/home/hoseung/Work/data/05427/'
nout = 187
halo_ok = 426


#----------------------------------------------------------------------
mstar_min = 5e9
# Only galaxies above this stellar mass at the final snapshot are considered.

mk_gal_rscale = 2.0 # unit of Rvir,galaxy
rscale = 1.5
npix=800
rscale_lambda = 3.0 # Reff unit.
npix_lambda = int(10 * rscale_lambda)
lmax = 19
ptypes=["star id pos mass vel time metal", "dm id pos mass vel"]

## halo part -----------------------------------------------------------
m_halo_min = 5e9 # minimum galaxy mass above which galaxies are searched for. 
dir_out = wdir + 'catalog_GM/'

# optional parameters ----------------------------------------------------
do_galaxy_plot=False
lambda_plot = False

info = load.info.Info(nout=nout, base=wdir, load=True)

# Do I really need halo bricks?
hh = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info, load=True, is_gal=is_gal)
hind = hh.data['id'] == halo_ok
h = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info, is_gal=is_gal)
h.derive_from(hh, hind)
halodata = h.data
region = smp.set_region(xc=halodata['x'],
                    yc=halodata['y'],
                    zc=halodata['z'],
                    radius = halodata['rvir'])


s = load.sim.Sim(nout=nout, base=wdir, setup=True, region = region)
#s.set_ranges(region = region)
s.add_part(ptypes, load=True, fortran=True)

if hydro:
    s.add_hydro(load=True, lmax=lmax)
    cell_all = s.hydro.cell
else:
    cell_all = None

star_all = s.part.star
dm_all = s.part.dm

keywords = dict(galaxy_plot_dir='./',
            rscale = mk_gal_rscale,
            verbose=False, rscale_lambda=rscale_lambda,
            npix_lambda=npix_lambda, galaxy_plot = do_galaxy_plot,
            mstar_min=mstar_min)


from queue import Queue
out_q = Queue()

gal = mk_gal(h.data, s.info, 0, 12345, **keywords)

print("----------Done---------")

Ranges = [[array([ 0.50537813], dtype=float32), array([ 0.50605011], dtype=float32)], [array([ 0.31046501], dtype=float32), array([ 0.31113693], dtype=float32)], [array([ 0.30288309], dtype=float32), array([ 0.30355501], dtype=float32)]]

No AMR instance,
Loading one...
An AMR instance is created

Updating info.cpus
An AMR instance is created

Updating info.cpus
 Simulation set up.
Types of particles you want to load are:  ['star id pos mass vel time metal', 'dm id pos mass vel']
No AMR instance,
Loading one...
An AMR instance is created

Updating info.cpus
No info._set_cpus attribute??
A particle instance is created

Loading by fortran module
Fortran-reading done
An Hydro instance is created

 >>> working resolution (lmax) = 19
Making a galaxy: 426
Number of stars 132605
galaxy::mk_gal::m_sun  {:.3e} 61004270394.4
Resampling 132605 values to a 5 by 5 by 5 grid
Updating field vals
Updating field vals
Updating field vals
Updating field vals
Updating field vals
Updating field vals
Updati

In [95]:
from draw import pp
from matplotlib.colors import LogNorm
# npix = round(gal.nstar**(1/3) * 3) # to maintain roughly same pixel density. 
# Or, constant physical scale

rscale = 3

npix = round(gal.reff * rscale) * 4 # 5 * reff = Rgal in most case, 4 pixels in 1 kpc.
region = smp.set_region(xc=0, yc=0, zc=0, radius = gal.reff * rscale)  
data = pp.den2d(gal.star['x'], gal.star['y'], gal.star['z'], gal.star['m'], \
              npix, region=region, cic=True, norm_integer=True)

from Cappellari import mge
import utils.util
utils.util.reimport(mge)
fig, ax = plt.subplots(1)


nbin = 5 * rscale #  =15 

eps_arr = np.zeros(nbin)
mjr_arr = np.zeros(nbin)
pa_arr = np.zeros(nbin)
xpos_arr = np.zeros(nbin)
ypos_arr = np.zeros(nbin)

l_img = 2 * region['radius'] # in kpc.

for i in range(15):
    f = mge.find_galaxy.find_galaxy(data, quiet=False, plot=True, mask_shade=False, fraction=0.005 + 0.01 * i)
    mjr_arr[i] = f.majoraxis * 3.5 / npix * l_img
    eps_arr[i] = f.eps
    pa_arr[i] = f.theta
    xpos_arr[i] = f.xpeak
    ypos_arr[i] = f.ypeak
    
    #print(f.eps, f.theta, f.majoraxis)
plt.show()

0.00876614173823 115.986129977 0.0342292509109 115.991414173
Resampling 100751 values to a 116 by 116 by 1 grid
Updating field vals
Updating field vals
Updating field vals
Updating field vals
minmax field after crop and converting into physical unit 570.726203118 657090919.703
 Pixels used: 68
 Peak Img[j, k]: 56 60
 Mean (j, k): 56.37 60.91
 Theta (deg): 24.4
 Astro PA (deg): 65.6
 Eps: 0.174
 Sigma along major axis (pix): 2.4
 Pixels used: 202
 Peak Img[j, k]: 56 60
 Mean (j, k): 56.43 60.98
 Theta (deg): 16.8
 Astro PA (deg): 73.2
 Eps: 0.223
 Sigma along major axis (pix): 4.0
 Pixels used: 337
 Peak Img[j, k]: 56 60
 Mean (j, k): 56.36 60.80
 Theta (deg): 13.4
 Astro PA (deg): 76.6
 Eps: 0.262
 Sigma along major axis (pix): 5.1
 Pixels used: 470
 Peak Img[j, k]: 56 60
 Mean (j, k): 56.47 60.75
 Theta (deg): 14.5
 Astro PA (deg): 75.5
 Eps: 0.297
 Sigma along major axis (pix): 6.0
 Pixels used: 606
 Peak Img[j, k]: 56 60
 Mean (j, k): 56.43 60.78
 Theta (deg): 13.0
 Astro PA (deg): 

In [ ]:
gal.rscale_lambda

In [63]:
def general_ellipse(x, y, a, b, xoff, yoff, theta):
    return np.square((x - xoff) * np.cos(theta) + (y - yoff)*np.sin(theta)) / a**2 + \
    np.square((x - xoff) * np.sin(theta) - (y - yoff)*np.cos(theta)) / b**2

def cal_lambda_better(self, npix_lambda=5,
                 r=0.5,
                 rscale=3.0,
                 method=2,
                 verbose=False):
    import numpy as np
    from Cappellari import mge

    # First, measure photometric properties
    nbin = npix * rscale #  =15 

    npix_img = round(self.reff * rscale) * 4 # 5 * reff = Rgal in most case, 4 pixels in 1 kpc.
    region = smp.set_region(xc=0, yc=0, zc=0, radius = self.reff * rscale)  
    data = pp.den2d(self.star['x'], self.star['y'], self.star['z'], self.star['m'], \
          npix_img, region=region, cic=True, norm_integer=True)

    arr = photometry(data, nbin, npix_img, 2 * region['radius'])
    #arr.mjr = arr.mjr * 3.5 / npix_img *  # in kpc.
    
    
    
    # velocity map & velocity dispersion map.
    a_max = arr.mjr[-1]
    b_max = a_max * arr.eps[-1]
#    ind_ok = np.where(np.squar())
    sigmap, vmap = sigmap_vmap(self, npix_img, ind_ok=None)
 
    x = gal.star['x']
    y = gal.star['y']
    #z = gal.star['z']
    
    # 
    for i in range(nbin):
        a = arr.mjr[i]
        b = arr.mjr[i] * arr.eps[i]
        xoff = arr.xpeak[i]
        yoff = arr.ypeak[i]
        theta = arr.theta[i]
                       
        q = general_ellipse(x, y, a, b, xoff, yoff, theta)
        ind = np.where((0.9 < q) & (q < 1))[0]
        print(len(ind))
    
    
    

In [9]:
plt.plot(mjr_arr)
plt.show()

### 2-D Gaussian fitting

In [3]:
from scipy import optimize
from pylab import *

def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = indices(data.shape)
    x = (X*data).sum()/total
    y = (Y*data).sum()/total
    col = data[:, int(y)]
    width_x = sqrt(abs((arange(col.size)-y)**2*col).sum()/col.sum())
    row = data[int(x), :]
    width_y = sqrt(abs((arange(row.size)-x)**2*row).sum()/row.sum())
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [32]:
# Create the gaussian data (with a bit of noise:

#Xin, Yin = np.mgrid[0:401, 0:401]
#data = gaussian(3, 200, 200, 40, 80)(Xin, Yin) + np.random.random(Xin.shape)
#matshow(data, cmap=cm.jet)
plt.imshow(data, cmap=plt.cm.jet)
#plt.show()
params = fitgaussian(data)
fit = gaussian(*params)
plt.contour(fit(*indices(data.shape)), cmap=cm.Greys)
ax = gca()
(height, x, y, width_x, width_y) = params

text(0.95, 0.05, """
x : %.1f
y : %.1f
width_x : %.1f
width_y : %.1f """ %(x, y, width_x, width_y),
        fontsize=16, color='w', horizontalalignment='right',
        verticalalignment='bottom', transform=ax.transAxes)


title('Fitting a 2-d Gaussian', fontsize=16, color='blue')

show()

#### Find PA

In [30]:
def main():
    xbar, ybar, cov = intertial_axis(data)
    fig, ax = plt.subplots()
    ax.imshow(data)
    plot_bars(xbar, ybar, cov, ax)
    plt.show()

def raw_moment(data, iord, jord):
    nrows, ncols = data.shape
    y, x = np.mgrid[:nrows, :ncols]
    data = data * x**iord * y**jord
    return data.sum()

def intertial_axis(data):
    """Calculate the x-mean, y-mean, and cov matrix of an image."""
    data_sum = data.sum()
    m10 = raw_moment(data, 1, 0)
    m01 = raw_moment(data, 0, 1)
    x_bar = m10 / data_sum
    y_bar = m01 / data_sum
    u11 = (raw_moment(data, 1, 1) - x_bar * m01) / data_sum
    u20 = (raw_moment(data, 2, 0) - x_bar * m10) / data_sum
    u02 = (raw_moment(data, 0, 2) - y_bar * m01) / data_sum
    cov = np.array([[u20, u11], [u11, u02]])
    return x_bar, y_bar, cov

def plot_bars(x_bar, y_bar, cov, ax):
    """Plot bars with a length of 2 stddev along the principal axes."""
    def make_lines(eigvals, eigvecs, mean, i):
        """Make lines a length of 2 stddev."""
        std = np.sqrt(eigvals[i])
        vec = 2 * std * eigvecs[:,i] / np.hypot(*eigvecs[:,i])
        x, y = np.vstack((mean-vec, mean, mean+vec)).T
        return x, y
    mean = np.array([x_bar, y_bar])
    eigvals, eigvecs = np.linalg.eigh(cov)
    ax.plot(*make_lines(eigvals, eigvecs, mean, 0), marker='o', color='white')
    ax.plot(*make_lines(eigvals, eigvecs, mean, -1), marker='o', color='red')
    ax.axis('image')

if __name__ == '__main__':
    main()

In [30]:
gal.reff

14.103231652528891

In [31]:
gal.region['radius']

70.516158262644453

In [16]:
data.data

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [29]:
f.majoraxis

33.606749273638279